# Aula 10 &mdash; Visualização interativa de dados com Bokeh

Renato Vimieiro

rv2 {em} cin.ufpe.br

setembro 2017

## Introdução

Na aula passada vimos como desenhar gráficos estáticos com Matplotlib e Seaborn usando a interface de Pandas. Como discutimos, a análise exploratória de dados pode se beneficiar dos recursos gráficos disponíveis nas bibliotecas mais recentes. A capacidade de poder interagir com a visualização e, dessa forma, explorar de forma mais natural os dados disponíveis facilita alcançar os objetivos da análise exploratória de dados.

Utilizaremos a biblioteca [Bokeh](http://bokeh.pydata.org/en/0.12.5/) como meio para a construção dos gráficos. A biblioteca possui três níveis de acesso (três interfaces) que vão do mais básico ao mais avançado. Os níveis são:

- ~~**básica** [bokeh.charts](http://bokeh.pydata.org/en/0.12.5/docs/user_guide/concepts.html#bokeh-charts). Esta interface contém métodos para o desenho de gráficos básicos com algumas funções predeterminadas para interação. O uso é mais fácil, porém a configuração dos gráficos é limitada.~~ A partir da versão 0.12.9 (divulgada em set/2017), Bokeh passou a não oferecer mais suporte a essa interface. A sugestão deles é usar diretamente o módulo `bkcharts` (que não é mais mantido). Em `bkcharts`, há uma nova sugestão para que seja usada a biblioteca [HoloViews](http://holoviews.org). Dadas as instabilidades das bibliotecas, seguiremos usando a versão 0.12.5 (mar/2017) sobre a qual as aulas foram criadas.
- **intermediária** [bokeh.plotting](http://bokeh.pydata.org/en/0.12.5/docs/user_guide/concepts.html#bokeh-plotting). Essa interface fornece uma gramática para construção dos gráficos. Ela se assemelha a alguns recursos de Matplotlib por permitir criar uma figura e seguir introduzindo elementos visuais a ela. Em outras palavras, ela permite um controle maior de como o gráfico será construído. Contudo, o usuário despenderá mais tempo para criar suas visualizações.
- **avançada** [bokeh.models](http://bokeh.pydata.org/en/0.12.5/docs/user_guide/concepts.html#bokeh-models). Esta interface expõe todos os detalhes ao usuário, permitindo que ele tenha total controle dos elementos a serem exibidos. Bokeh, de fato, é composta de dois módulos. O primeiro módulo, BokehJS, é implementada em Javascript e é a responsável por gerar e exibir os gráficos no navegador. A interface `bokeh.models` mapeia esses recursos da biblioteca base em Python. Isso é o quê permite ao usuário ter mais controle sobre o que é exibido. Raramente os usuários utilizarão essa interface, a menos que seja necessário ajustes muito finos na visualização.

Embora o mais natural seja utilizar a interface intermediária para se ter mais controle sobre as visualizações, iniciaremos com a interface básica para ilustrar a construção dos gráficos básicos vistos na aula passada com Bokeh.

In [1]:
import pandas as pd
import numpy as np

import bokeh.charts as charts
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import Range1d, NumeralTickFormatter, FixedTicker, Legend
from bokeh.palettes import Set1_7, RdBu5, RdBu11

output_notebook()

Loading BokehJS ...

In [2]:
expvida = pd.read_csv("../2017.1/data/banco_mudial/life_expectancy.csv",index_col=0,
                      skiprows=[0,1,2],)
expvida = expvida.drop(['Indicator Name', 'Indicator Code', 
                        'Country Code'],axis=1).dropna(axis=1,how='all')
gdp = pd.read_csv("../2017.1/data/banco_mudial/gdp.csv",index_col=0,
                      skiprows=[0,1,2],)
gdp = gdp.drop(['Indicator Name', 'Indicator Code', 
                'Country Code', '2015'],axis=1).dropna(axis=1,how='all')

gdplong = pd.melt(gdp.reset_index(),id_vars='Country Name')
evidalong = pd.melt(expvida.reset_index(),id_vars='Country Name')

## Gráficos básicos com a interface bokeh.charts

Bokeh possui funções pré-definidas para plotar os seguintes gráficos:

- [Area](http://bokeh.pydata.org/en/0.12.5/docs/reference/charts.html#area)
- [Barras](http://bokeh.pydata.org/en/0.12.5/docs/reference/charts.html#bar)
- [Boxplot](http://bokeh.pydata.org/en/0.12.5/docs/reference/charts.html#boxplot)
- [Grafos](http://bokeh.pydata.org/en/0.12.5/docs/reference/charts.html#chord)
- [Heatmap](http://bokeh.pydata.org/en/0.12.5/docs/reference/charts.html#heatmap)
- [Histograma](http://bokeh.pydata.org/en/0.12.5/docs/reference/charts.html#histogram)
- [Linha](http://bokeh.pydata.org/en/0.12.5/docs/reference/charts.html#line)
- [Dispersão](http://bokeh.pydata.org/en/0.12.5/docs/reference/charts.html#scatter)
- [Escada](http://bokeh.pydata.org/en/0.12.5/docs/reference/charts.html#step)

De uma forma geral, os gráficos esperam como parâmetros uma tabela de dados (data frame) e a coluna dessa tabela contendo os valores a serem plotados. Os tipos mais comuns vistos na aula passada ainda esperam a especificação da coluna que será usada para definir as séries de dados, e a coluna especificando como os dados são agrupados. Isso sugere implicitamente que o formato da tabela de dados é longo. Dessa forma, em alguns casos é mais conveniente converter os dados para esse formato antes de começar a explorá-los.

Vamos começar pelos mesmos exemplos da aula passada. Depois seguiremos para os outros tipos não abordados anteriormente.

#### Gráfico de barras

In [3]:
BRICS = ["Brazil", "Russian Federation","India", 'China', 
         'South Africa', 'Euro area', 'United States']

In [4]:
bar = charts.Bar(gdp.loc[BRICS], values='2010', 
                 plot_height=400, legend=None, color='#0067A9')
show(bar)

No exemplo a seguir, vamos comparar o PIB dos BRICS com a zona do euro e EUA desde 1990. Precisamos filtrar os dados para plotá-los. Depois usaremos os nomes dos países para agrupar os dados e os anos para definir as séries. Dessa forma compararemos os dados dos países ao longo dos anos. Porém, se quiséssemos comparar a evolução do PIB nos anos para cada país, deveríamos agrupar os dados por anos e definir os nomes dos países como séries.

In [5]:
dados = gdplong[(gdplong['Country Name'].isin(BRICS)) & \
                (gdplong['variable'].isin(['1990','1995','2000','2005','2010','2014']))]
bar = charts.Bar(dados, values='value', group='Country Name', label='variable',
                 color=Set1_7,plot_height=400,plot_width=800)
show(bar)

Uma funcionalidade interessante dos gráficos básicos de Bokeh é a computação de funções de agregação dos dados. Assim, ao contrário dos gráficos com Matplotlib em Pandas, podemos passar a tabela de dados para a função normalmente e especificar a forma como os dados devem ser sumarizados. De fato, esse parâmetro é obrigatório e possui como valor padrão a soma dos dados em cada séries. As possíveis funções de agregação são: sum, mean, count, nunique, median, min, max. Elas devem ser passadas como strings.

A seguir construimos um gráfico em que mostramos a expectativa de vida média dos países dos BRICS.

In [6]:
bar = charts.Bar(evidalong[evidalong['Country Name'].isin(BRICS)], 
                 values='value', label='Country Name', agg='mean',
                 plot_height=400, legend=None)
show(bar)

#### Histogramas

A função de histogramas possui uma importante diferença em relação à de gráfico de barras. Ela não permite que os dados tenham valores nulos. Dessa forma, devemos tratar os dados ausentes antes de passar a matriz de dados para a função.

In [7]:
hist = charts.Histogram(expvida['1960'].dropna(), bins=20, plot_height=400)
hist.x_range=Range1d(0,100)
show(hist)

Assim como fizemos em Pandas+Matplotlib, podemos plotar vários histogramas numa mesma figura para compararmos as distribuições. Para fazê-lo, temos que informar qual é a coluna com os valores e a coluna definindo as séries, tal como fizemos com o gráfico de barras. Devemos informar também a coluna que determina as cores das séries, caso contrário todas serão plotadas com uma única cor.

Outro comentário em relação à função de histogramas é que ela permite exibir os valores absolutos ou a frequência relativa. Para exibir a frequência relativa, basta passar o parâmetro `density` com valor verdadeiro.

In [8]:
dados = evidalong[evidalong.variable.isin(['1960','1980','2000','2014'])].dropna()
hist = charts.Histogram(dados, values='value',label='variable', color='variable',
                        fill_alpha=0.3, density=True,
                        bins=20, plot_height=400)
hist.x_range=Range1d(0,100)
show(hist)

Infelizmente Bokeh não conta com uma função para desenhar gráficos de densidade nem compô-los com o histograma, tal como fizemos com Seaborn. É possível fazê-lo, porém com a interface intermediária. Quando apresentarmos a interface intermediária, voltaremos a esse exemplo.

#### Boxplot

A criação de boxplots é bem similar a dos gráficos de barra. Devemos especificar a coluna de valores e a coluna determinando as séries de dados.

In [9]:
dados = evidalong[evidalong['Country Name'].isin(BRICS)]
box = charts.BoxPlot(dados,values='value',label='Country Name', 
                     legend=None, plot_height=500, 
                     color='Country Name')
                     #color='#dddddd')
show(box)

Bokeh também possui uma interface bem amigável para combinar vários gráficos em um painel. As diversas funções para especificar a diagramação da exibição estão no submódulo `bokeh.layouts`.

In [10]:
from bokeh.layouts import row

In [11]:
dados = evidalong[evidalong['Country Name'].isin(BRICS)]
box = charts.BoxPlot(dados,values='value',label='Country Name', 
                     legend=None, plot_height=400, plot_width=450, 
                     color='#dddddd', tools='')
dados = gdplong[gdplong['Country Name'].isin(BRICS)]
box1 = charts.BoxPlot(dados,values='value',label='Country Name', 
                     legend=None, plot_height=400, plot_width=450, 
                      color='#dddddd', tools='')
show(row([box,box1]))

#### Gráficos de dispersão

Para os gráficos de dispersão, devemos especificar as colunas que representam os eixos x e y nos dados. Ambas as colunas devem ser numéricas, naturalmente. 

Em nosso exemplo, vamos criar um novo conjunto de dados formado pela expectativa de vida dos países em 1960 e o eixo x será simplesmente o índice da linha.

In [12]:
dados = pd.DataFrame({'x':np.arange(1,expvida['1960'].shape[0]+1), 'y':expvida['1960']})
scatter = charts.Scatter(dados,x='x',y='y',plot_height=400)
show(scatter)

Vamos criar um outro exemplo para explorar os dados do PIB dos BRICS ao longo dos anos.

In [13]:
dados = gdplong.loc[gdplong['Country Name'].isin(BRICS[:-2])].dropna()
dados[['variable','value']].astype(np.number,inplace=True)
scatter = charts.Scatter(dados,x='variable',y='value',
                         plot_height=400, plot_width=900,
                         color='Country Name')
show(scatter)

#### Gráfico de linha

O gráfico de linha segue a mesma lógica do gráfico de dispersão. Devemos informar o conjunto de dados e as colunas representando os eixos x e y do gráfico. Para exemplificar, vamos recriar o exemplo da aula passada, exibindo a variação do PIB e expectativa de vida dos BRICS.

In [14]:
dados = gdplong.loc[gdplong['Country Name'].isin(BRICS[:-2])].dropna()
dados.variable = dados.variable.astype(int)
dados.value = dados.value.astype(np.number)
line = charts.Line(dados,x='variable',y='value',
                         plot_height=400, plot_width=450,
                         color='Country Name')

dados = evidalong.loc[evidalong['Country Name'].isin(BRICS[:-2])].dropna()
dados.variable = dados.variable.astype(int)
dados.value = dados.value.astype(np.number)
line1 = charts.Line(dados,x='variable',y='value',
                         plot_height=400, plot_width=450, legend='bottom_right',
                         color='Country Name')
show(row([line,line1]))

In [15]:
line.legend.background_fill_alpha = 0
line1.legend.background_fill_alpha = 0
line.yaxis[0].formatter = NumeralTickFormatter(format="$0,0a")

In [16]:
show(row([line,line1]))

#### Gráficos de escada

O gráfico de linha nos exemplos acima pode ser um tanto enganoso, uma vez que os dados foram coletados anualmente. O gráfico de linha passa a impressão de uma variação contínua. Isso não acontece de fato. Podemos usar um gráfico de escada para representar melhor as alterações ocorridas. Bokeh possui uma função para isso. A sintaxe é exatamente igual à dos gráficos de linha.

In [17]:
dados = evidalong.loc[(evidalong['Country Name'].isin(BRICS[:-2])) & 
                     (evidalong['variable'].isin([str(a) for a in range(1960,2015,10)]))].dropna()
dados.variable = dados.variable.astype(int)
dados.value = dados.value.astype(np.number)
escada = charts.Step(dados,x='variable',y='value',
                         plot_height=400, plot_width=450, legend='bottom_right',
                         color='Country Name')
escada.legend.background_fill_alpha = 0
show(escada)

#### Heatmap

Outro tipo de gráfico que é bastante útil para visualização de dados tabulares é o heatmap. Por exemplo, se quisermos ver 

In [18]:
wine = pd.read_csv("http://mlr.cs.umass.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv",
                   sep=';')

In [19]:
wine = wine.groupby('quality').mean().apply(lambda x: (x-x.mean())/x.std())
wine.reset_index(inplace=True)

In [20]:
wine.head()

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,3,1.562498,0.721505,-0.011776,0.630726,1.127678,1.939284,1.834338,0.948237,-0.476842,-0.678123,-0.598607
1,4,0.238445,1.835414,-1.288769,-0.935840,0.685628,-1.443218,-0.559591,0.530721,-0.575686,-0.543791,-0.819616
2,5,-0.311585,-0.003789,0.054661,1.467577,0.837981,0.032448,0.793992,1.208496,-0.876510,-0.045217,-1.214025
3,6,-0.582566,-0.966083,0.069646,0.674329,0.172161,-0.055761,0.062025,0.313932,-0.453318,0.686220,-0.334181
4,7,-0.872263,-0.914937,-0.428839,-0.440145,-0.567060,-0.227902,-0.568273,-0.723441,0.088350,1.671893,0.575544


In [21]:
dados = pd.melt(wine,id_vars='quality')
dados.quality = dados.quality.astype(str)
heatmap = charts.HeatMap(dados,x='quality',y='variable',values='value',
                         stat='mean',legend=False, plot_height=400,plot_width=400,
                        hover_tool=True, color=RdBu11)
#l = heatmap.legend
#l.location = (100,-30)
#heatmap.add_layout(Legend(items=l[0].items,location=(100,-30)))
show(heatmap)